In [ ]:
# This code was modified from Hack Oregon's Neighborhood Development project
# https://github.com/hackoregon/neighborhoods-2018/tree/master/notebooks/campsites
# Many thanks to Hasan for collaborating on the data prep through Hack Oregon

In [8]:
import pyproj
from pyproj import transform

In [9]:
import pandas as pd
import numpy as np

In [10]:
# http://www.spatialreference.org/ref/epsg/3646/
or_north_plain = pyproj.Proj(init='EPSG:3646', preserve_units=True)

In [11]:
def load_csv(fname):
    falsies = ['No', 'NO','no', ]
    trusies = ['YES', 'Yes', 'yes']
    df = pd.read_csv(fname,
                     true_values=trusies,
                     false_values=falsies,
                     parse_dates=[0],
                     low_memory=False,
                     )
    return df
    

In [12]:
def transform_coordinates(df, projection):
    # 0 values in X and Y columns are invalid - replace with NaN so we can fix erronous projections
    df.X.replace(0, np.nan, inplace=True)
    df.Y.replace(0, np.nan, inplace=True)
    X = np.array(df.X)
    Y = np.array(df.Y)
    
    lons, lats = projection(X, Y, inverse=True)
    # Projections that don't make sense return 1e+30 rather than erroring.
    # We will replace these with NaNs
    lons[lons == 1e+30] = np.nan
    lats[lats == 1e+30] = np.nan
    df['lon'] = lons
    df['lat'] = lats
    # Get rid of old columns
    df.drop(columns=['X', 'Y'], inplace=True)
    
    return df

In [13]:
# def transform_coordinates_to_epsg4326(df, projection, outprojection):
#     # 0 values in X and Y columns are invalid - replace with NaN so we can fix erronous projections
#     df.X.replace(0, np.nan, inplace=True)
#     df.Y.replace(0, np.nan, inplace=True)
#     X = np.array(df.X)
#     Y = np.array(df.Y)
    
#     lons, lats = projection(X, Y, inverse=True)
#     # Projections that don't make sense return 1e+30 rather than erroring.
#     # We will replace these with NaNs
#     lons[lons == 1e+30] = np.nan
#     lats[lats == 1e+30] = np.nan
#     df['lon'] = lons
#     df['lat'] = lats
#     # Transform to EPSG 4326
#     (df['lon'], df['lat']) = transform(projection, outprojection, (df['lon'], df['lat']))
#     # Get rid of old columns
#     df.drop(columns=['X', 'Y'], inplace=True)
    
#     return df

In [14]:
def cleanse_data(df):
    # Get the first number and use that as the new value
    df['num_campers'] = df['How many campers would you estimate are there?'].str.extract(r'(\d+)', expand=False)
#    df.drop(columns=['How many campers would you estimate are there?'], inplace=True)
    return df

#This needs to be fixed using the code below. Will do this later.

In [17]:
CSV_FILES = ['../reports_vehicle_filter_final.csv']

In [18]:
df = load_csv(CSV_FILES[0])

In [21]:
df.head()
print("df is {}".format(len(df['lat'])))
df.to_csv("test_df.csv")

df is 38870


In [23]:
df.drop('Unnamed..0', axis=1, inplace=True)
df.head()

,Date.Created,Date.Received,Item.ID,Status,Follow.Ups,Where.is.the.campsite.concern.located..Please.provide.an.address.if.possible..If.no.address.is.available..please.provide.a.complete.description.of.the.site.s.location.,If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...User.Specified.,If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...System.Verified.,Address.ID,Property.ID,...,Structures.or.tents.present,Excessive.trash.and.or.biohazards,Damage.to.the.environment,lon,lat,coordinates,index_right,NAME,SUM_AREA,SUM_SqMile
0,12/6/15 20:09,NaN,937188,NaN,NaN,SE 11th and Alder,SE 11th and Alder,SE 11TH AVE & SE ALDER ST,NaN,NaN,...,True,TRUE,FALSE,-122.654654,45.517920,POINT (-122.6546537 45.51791996),33,BUCKMAN,32168898.53,1.153900
1,12/8/15 14:19,NaN,938313,NaN,NaN,SW 18th Ave and Mill St. Terrace,SW 18th Ave & Mill St Terrace,SW 10TH AVE & SW MILL ST,NaN,NaN,...,True,TRUE,TRUE,-122.685714,45.513348,POINT (-122.6857142 45.51334834),85,DOWNTOWN,26280457.15,0.942682
2,12/14/15 16:37,NaN,940538,NaN,NaN,Sellwood park,Sellwood park,7951 SE 7TH AVE,NaN,NaN,...,True,TRUE,FALSE,-122.660276,45.467760,POINT (-122.660276 45.46775984),262,SELLWOOD-MORELAND IMPROVEMENT LEAGUE,58065718.62,2.082821
3,12/17/15 13:26,NaN,941670,NaN,NaN,9th ave and NW Naito,NW 5TH AVE AND NAITO,NW 9TH AVE & NW NAITO PKWY,NaN,NaN,...,True,TRUE,FALSE,-122.679752,45.532840,POINT (-122.6797517 45.5328399),230,PEARL,12417559.13,0.445419
4,12/20/15 12:39,NaN,942320,NaN,NaN,SE 94th. between Holgate and Pardee,NaN,NaN,NaN,NaN,...,False,TRUE,FALSE,-122.567034,45.488839,POINT (-122.5670341 45.4888386),166,LENTS,89541435.25,3.211857


In [34]:
df.columns
len1 = len(df['Date.Created'])
print(len1)
#len2 = len(df2['Date Created'])
#len3 =len(dfc['Date Created'])

#print(len1+len2)
#print(len1, len2, len3)

#Looks like they concat'd successfully
#dfc.to_csv("test_dfc.csv")

38870


In [37]:
dfc = df
df.columns

Index(['Date.Created', 'Date.Received', 'Item.ID', 'Status', 'Follow.Ups',
       'Where.is.the.campsite.concern.located..Please.provide.an.address.if.possible..If.no.address.is.available..please.provide.a.complete.description.of.the.site.s.location.',
       'If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...User.Specified.',
       'If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...System.Verified.',
       'Address.ID', 'Property.ID', 'State.ID', 'City',
       'How.long.has.the.site.been.there.', 'Is.the.site.occupied.',
       'How.many.campers.would.you.estimate.are.there.',
       'Are.children.present.', 'Are.dogs.present.',
       'Does.anyone.seem.to.be.medically.fragile.',
       'Are.there.tents.or.structures.there..If.so..how.many.',
       'Is.this.a.veh

In [38]:
s = dfc['How.many.campers.would.you.estimate.are.there.']
s.head(40)


0                                             NaN
1                                               2
2                                              4?
3                                         Unknown
4                                               4
5                                               3
6                                             NaN
7                                             NaN
8                                              10
9                                       15 people
10                2 or 3 RVs parked and occupied.
11                                             15
12                                              2
13    Not sure, 2 or more, do not know the next 3
14                                             20
15                                             15
16                                             12
17                                            NaN
18                                              6
19                                            NaN


In [59]:
#Convert any numbers that are written out into integers
islongint = s.str.match('one', na=False)
islongint
#Locate where the 'one' text is
#s.loc[s.str.match('one') == True]
#islongint.loc[30]

#dfc.loc[islongint, 'How many campers would you estimate are there?'] = '1'
#dfc.loc[731]

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
38840    False
38841    False
38842    False
38843    False
38844    False
38845    False
38846    False
38847    False
38848    False
38849    False
38850    False
38851    False
38852    False
38853    False
38854    False
38855    False
38856    False
38857    False
38858    False
38859    False
38860    False
38861    False
38862    False
38863    False
38864    False
38865    False
38866    False
38867    False
38868    False
38869    False
Name: How.many.campers.would.you.estimate.are.there., Length: 38870, dtype: bool

In [60]:
islongint = s.str.match('two', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '2'

islongint = s.str.match('three', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '3'

islongint = s.str.match('four', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '4'

islongint = s.str.match('five', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '5'

islongint = s.str.match('six', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '6'

islongint = s.str.match('seven', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '7'

islongint = s.str.match('eight', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '8'

islongint = s.str.match('nine', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '9'

islongint = s.str.match('r"ten\s"', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '10'

islongint = s.str.match('fifteen', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '15'

islongint = s.str.match('twenty', na=False)
dfc.loc[islongint, 'How many campers would you estimate are there?'] = '20'


In [61]:
s.loc[s.str.match('one') == True]

Series([], Name: How.many.campers.would.you.estimate.are.there., dtype: object)

In [70]:
# Number of rows without any digit in the 'How many campers...' column
s.str.extract(r"(\d+)", expand=False).isnull().sum()

28828

In [72]:
# First pull out any rows with months in the string, replace with NaN (K)
#(\d+-Apr$)   This will pull out specifically the month of April. 
#(\d{1,2})-([A-Z][a-z]{2}) pulls out all months. 

isbaddate = s.str.match(r"(\d{1,2})-([A-Z][a-z]{2})", na=False)
isbaddate

0        False
1        False
2        False
3        False
4        False
5        False
6        False
7        False
8        False
9        False
10       False
11       False
12       False
13       False
14       False
15       False
16       False
17       False
18       False
19       False
20       False
21       False
22       False
23       False
24       False
25       False
26       False
27       False
28       False
29       False
         ...  
38840    False
38841    False
38842    False
38843    False
38844    False
38845    False
38846    False
38847    False
38848    False
38849    False
38850    False
38851    False
38852    False
38853    False
38854    False
38855    False
38856    False
38857    False
38858    False
38859    False
38860    False
38861    False
38862    False
38863    False
38864    False
38865    False
38866    False
38867    False
38868    False
38869    False
Name: How.many.campers.would.you.estimate.are.there., Length: 38870, dtype: bool

In [32]:
#Check out loc, iloc, and ix

In [73]:
dfc.loc[isbaddate, 'How.many.campers.would.you.estimate.are.there.'] = np.nan




In [77]:
dfc.head()


,Date.Created,Date.Received,Item.ID,Status,Follow.Ups,Where.is.the.campsite.concern.located..Please.provide.an.address.if.possible..If.no.address.is.available..please.provide.a.complete.description.of.the.site.s.location.,If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...User.Specified.,If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...System.Verified.,Address.ID,Property.ID,...,Excessive.trash.and.or.biohazards,Damage.to.the.environment,lon,lat,coordinates,index_right,NAME,SUM_AREA,SUM_SqMile,How many campers would you estimate are there?
0,12/6/15 20:09,NaN,937188,NaN,NaN,SE 11th and Alder,SE 11th and Alder,SE 11TH AVE & SE ALDER ST,NaN,NaN,...,TRUE,FALSE,-122.654654,45.517920,POINT (-122.6546537 45.51791996),33,BUCKMAN,32168898.53,1.153900,NaN
1,12/8/15 14:19,NaN,938313,NaN,NaN,SW 18th Ave and Mill St. Terrace,SW 18th Ave & Mill St Terrace,SW 10TH AVE & SW MILL ST,NaN,NaN,...,TRUE,TRUE,-122.685714,45.513348,POINT (-122.6857142 45.51334834),85,DOWNTOWN,26280457.15,0.942682,NaN
2,12/14/15 16:37,NaN,940538,NaN,NaN,Sellwood park,Sellwood park,7951 SE 7TH AVE,NaN,NaN,...,TRUE,FALSE,-122.660276,45.467760,POINT (-122.660276 45.46775984),262,SELLWOOD-MORELAND IMPROVEMENT LEAGUE,58065718.62,2.082821,NaN
3,12/17/15 13:26,NaN,941670,NaN,NaN,9th ave and NW Naito,NW 5TH AVE AND NAITO,NW 9TH AVE & NW NAITO PKWY,NaN,NaN,...,TRUE,FALSE,-122.679752,45.532840,POINT (-122.6797517 45.5328399),230,PEARL,12417559.13,0.445419,NaN
4,12/20/15 12:39,NaN,942320,NaN,NaN,SE 94th. between Holgate and Pardee,NaN,NaN,NaN,NaN,...,TRUE,FALSE,-122.567034,45.488839,POINT (-122.5670341 45.4888386),166,LENTS,89541435.25,3.211857,NaN


In [78]:
# Pull out any numbers in that string
temp = s.str.extractall(r"(?P<Numbers>\d+)")
temp


Numbers
      match        
1     0           2
2     0           4
4     0           4
5     0           3
8     0          10
9     0          15
10    0           2
      1           3
11    0          15
12    0           2
13    0           2
      1           3
14    0          20
15    0          15
16    0          12
18    0           6
20    0          40
21    0          10
22    0           5
23    0          15
24    0           3
26    0          12
28    0          50
29    0           4
30    0          50
31    0           6
32    0          26
34    0          75
37    0           1
38    0           2
...             ...
38757 0           6
38759 0           2
38762 0           4
38764 0           2
38768 0           1
38769 0           2
38770 0           5
38773 0           2
38776 0           2
38783 0           2
38786 0           2
38791 0           0
38800 0           1
38805 0           2
38812 0          12
38821 0           6
38822 0           5
38823 0           5
38827 0           5
38829 0           3
38830 0           1
38834 0           4
38841 0           4
38848 0           3
38849 0           3
38850 0           1
38852 0           5
38854 0           1
38857 0          15
38858 0           1

[10800 rows x 1 columns]

In [79]:
temp2 = temp.Numbers.astype(float)

#temp2.loc[25].mean()
#THIS is correct.

In [80]:
temp2.index.names = ['record','match']
temp2.index.names

temp3 = temp2.groupby(level=['record']).mean()
temp3

record
1         2.0
2         4.0
4         4.0
5         3.0
8        10.0
9        15.0
10        2.5
11       15.0
12        2.0
13        2.5
14       20.0
15       15.0
16       12.0
18        6.0
20       40.0
21       10.0
22        5.0
23       15.0
24        3.0
26       12.0
28       50.0
29        4.0
30       50.0
31        6.0
32       26.0
34       75.0
37        1.0
38        4.0
40        2.0
42       10.0
         ... 
38757     6.0
38759     2.0
38762     4.0
38764     2.0
38768     1.0
38769     2.0
38770     5.0
38773     2.0
38776     2.0
38783     2.0
38786     2.0
38791     0.0
38800     1.0
38805     2.0
38812    12.0
38821     6.0
38822     5.0
38823     5.0
38827     5.0
38829     3.0
38830     1.0
38834     4.0
38841     4.0
38848     3.0
38849     3.0
38850     1.0
38852     5.0
38854     1.0
38857    15.0
38858     1.0
Name: Numbers, Length: 10042, dtype: float64

In [81]:
dfc2 = dfc.assign(number_of_campers=temp3)

In [83]:
dfc2.columns

Index(['Date.Created', 'Date.Received', 'Item.ID', 'Status', 'Follow.Ups',
       'Where.is.the.campsite.concern.located..Please.provide.an.address.if.possible..If.no.address.is.available..please.provide.a.complete.description.of.the.site.s.location.',
       'If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...User.Specified.',
       'If.having.difficulty.with.the.map..please.type.in.the.nearest.address.or.intersection..for.example..SW.Market.and.SW.14th..and.scroll.to.pinpoint.the.location...System.Verified.',
       'Address.ID', 'Property.ID', 'State.ID', 'City',
       'How.long.has.the.site.been.there.', 'Is.the.site.occupied.',
       'How.many.campers.would.you.estimate.are.there.',
       'Are.children.present.', 'Are.dogs.present.',
       'Does.anyone.seem.to.be.medically.fragile.',
       'Are.there.tents.or.structures.there..If.so..how.many.',
       'Is.this.a.veh

In [84]:
# dfc3 = dfc2.dropna(subset=['lon','lat', dfc2.columns[5], dfc2.columns[6], dfc2.columns[7]])
#NOTE: This is too strong; find the subset that is all 5 together (AND logic), not all 5 (OR logic)
#NOTE: Did not end up subsetting; will do this in the R notebook.
                           
# len(dfc3['number_of_campers'])
len(dfc2['number_of_campers'])

38870

In [87]:
# dfc2.to_csv('../data/onepointcontactreports/campsite_reports_processed_cols_nums.csv')
dfc2.to_csv('campsite_reports_with_num_campers_spatial_join.csv')

In [74]:
#np.mean(a, axis=0) 
#something like this will take the mean of each row for the indices and return a single-column df (or) array.
#from there, I can append the column to the dataframe. 
#from, I can filter out rows that have NaN either for the # of campers or for hte lat/long to get the final dataset.

In [73]:
#temp2 = temp.Numbers.astype(float)

#temp2.loc[27].mean()
#THIS is correct.

In [14]:
#result = cleanse_data(df)

In [15]:
#result

,Date Created,Item ID,"Where is the campsite/concern located? Please provide an address if possible. If no address is available, please provide a complete description of the site's location.","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (User Specified)","If having difficulty with the map, please type in the nearest address or intersection (for example, SW Market and SW 14th) and scroll to pinpoint the location. (System Verified)",X,Y,Address ID,Property ID,State ID,...,What was the make and model of the vehicle?,"What is the license plate, if available?",Repeated instances of overly aggressive behavior from campers,"Public intoxication and/or conspicuous drug use (If it is an emergency, please call 9-1-1.)",Campsite obstructs public right-of-way,Misuse of public spaces,Structures or tents present,Excessive trash and/or biohazards,Damage to the environment,num_campers
0,2017-03-31 21:26:00,1166167.0,along MUP North of SE Flavel and East of 92ndAve,NaN,NaN,0.000,0.000,NaN,NaN,NaN,...,NaN,NaN,False,False,True,True,True,Yes,Yes,5
1,2017-03-31 21:13:00,1166163.0,NaN,NaN,NaN,7671608.498,663720.642,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
2,2017-03-31 21:12:00,1166162.0,NaN,NaN,9690-9699 SE FLAVEL ST,7672165.664,663601.344,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
3,2017-03-31 20:42:00,1166158.0,NaN,NaN,12500-12899 NE SHAVER ST,7681419.287,693844.334,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
4,2017-03-31 20:40:00,1166157.0,NaN,NaN,12500-12899 NE SHAVER ST,7681264.838,693826.702,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
5,2017-03-31 20:37:00,1166156.0,NaN,NaN,12500-12899 NE SHAVER ST,7681237.336,693823.383,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
6,2017-03-31 20:34:00,1166154.0,NaN,NaN,12500-12899 NE SHAVER ST,7681516.553,693829.850,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
7,2017-03-31 20:16:00,1166152.0,NaN,NaN,NaN,7652801.131,674934.195,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
8,2017-03-31 19:41:00,1166149.0,Between NE 66th and 67th on NE Sacramento,2615 NE 58th,NaN,0.000,0.000,NaN,NaN,NaN,...,OR,NaN,False,False,False,True,True,No,Yes,NaN
9,2017-03-31 19:06:00,1166142.0,NaN,NaN,NaN,7672043.826,671335.119,NaN,NaN,NaN,...,NaN,NaN,False,False,False,False,False,No,No,NaN
